In [ ]:
import sys
from xml.etree import ElementTree as ET
import pandas as pd

save = False
csv_name = 'pit_entry.csv'
# Load OSM file

tree = ET.parse("map_osm/ims_map.osm")
root = tree.getroot()


# Dictionary to store node coordinates
node_coords = {}

# First, store the coordinates of all nodes
for node in root.iter('node'):
    node_id = node.get('id')
    lat = node.get('lat')
    lon = node.get('lon')
    name = node.get('name')
    node_coords[node_id] = (lat, lon, name)

# List to store ways containing 'IMS Oval' and their node coordinates
ims_oval_ways = []
key_words = ['ims oval', 'pit lane']
# Extract ways that contain 'IMS Oval'
for way in root.iter('way'):
    is_ims_oval = False
    for tag in way.iter('tag'):
        key = tag.get('k')
        value = tag.get('v')
        if any(keyword in value.lower() for keyword in key_words):
            is_ims_oval = True
            break
    if is_ims_oval:
        way_info = {
            'way_id': way.get('id'),
            'nodes': [],
            'way_name': None,
        }

        for tag in way.iter('tag'):
            key = tag.get('k')
            value = tag.get('v')
            if key == 'name':
                way_info['way_name'] = value

        for nd in way.iter('nd'):
            node_ref = nd.get('ref')
            if node_ref in node_coords:
                way_info['nodes'].append({
                    'node_id': node_ref,
                    'latitude': node_coords[node_ref][0],
                    'longitude': node_coords[node_ref][1],
                })
        ims_oval_ways.append(way_info)

# Create a dataframe to store the way information
df_nodes = pd.DataFrame(columns=['way_id', 'way_name','node_id', 'Latitude', 'Longitude'])
# data_frame = data_frame.append(ims_oval_ways, ignore_index=True)
df_nodes['way_id'] = [way['way_id'] for way in ims_oval_ways for node in way['nodes']]
df_nodes['way_name'] = [way['way_name'] for way in ims_oval_ways for node in way['nodes']]
df_nodes['node_id'] = [node['node_id'] for way in ims_oval_ways for node in way['nodes']]
df_nodes['Longitude'] = [node['longitude'] for way in ims_oval_ways for node in way['nodes']]
df_nodes['Latitude'] = [node['latitude'] for way in ims_oval_ways for node in way['nodes']]

# Filter the dataframe to only include nodes that contain certain keywords
# List of keywords
keywords = ["IMS Pit Lane", "Pit Entry"]
# Create a regex pattern that matches any of the keywords
pattern = '|'.join(keywords)
filtered_df = df_nodes[df_nodes['way_name'].str.contains(pattern, case=False, na=False)]
filtered_df


#### Visualization of GPS coordinates

In [ ]:
import matplotlib.pyplot as plt

# Ensure latitude and longitude are of float type
df_nodes['Latitude'] = filtered_df['Latitude'].astype(float)
df_nodes['Longitude'] = filtered_df['Longitude'].astype(float)

# Convert to numpy array
df_nodes_array = df_nodes.to_numpy()

pit_longitudes = []
pit_latitudes  = []
pit_through_longitudes = []
pit_through_latitudes  = []
for id, name in enumerate(df_nodes_array[:, 1]):
    if name is None:
        continue
    if "ims oval" in name.lower() and "pit entry" in name.lower():
        pit_longitudes.append(df_nodes_array[id, -1])
        pit_latitudes.append(df_nodes_array[id, -2])
    elif 'pit lane' in name.lower():
        pit_through_longitudes.append(df_nodes_array[id, -1])
        pit_through_latitudes.append(df_nodes_array[id, -2])



%matplotlib qt
# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(pit_longitudes, pit_latitudes, c='blue', marker='o', label='Pit Entry')
plt.scatter(pit_through_longitudes, pit_through_latitudes, c='green', marker='o', label='Pit Through')
plt.axis('equal')

# Set labels and title
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('GPS Coordinates of IMS Oval Nodes')
plt.legend()
plt.grid(True)
plt.show()


#### Data saving

In [ ]:
# Save the dataframe to a CSV file
if save:
    filtered_df = filtered_df[df_nodes['way_name'].str.contains("Pit Lane", case=False, na=False)]
    csv_name = 'pit_lane'
    filtered_df.to_csv('/home/jyning/dynamics/scripts/data_process/racetracks/ims/{}_gps.csv'.format(csv_name), index=False)
